In [ ]:
#config related imports
import tomli

#json parsing related imports
import json
import pandas as pd
from tqdm import tqdm
from pandas.io.json import json_normalize

#tweepy related imports
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
import pandas as pd

save_file = False

In [ ]:
with open("../config.toml", "rb") as f:
    config = tomli.load(f)

# Collect Legislrators Data

## Open legistrators social media json and convert to dataframe

In [ ]:
with open(config['input']['legislators_json']) as f:
  data = json.load(f)

df = json_normalize(data)

## Use tweepy to get the list of followers for each user


In [ ]:
user_count = 0
n=100
user_list = df['social.twitter_id'].dropna().unique()
lists_of_users=[user_list[i:i + n] for i in range(0, len(user_list), n)]

## Collect details on each of the users

In [ ]:
d_df = pd.DataFrame(columns = ['twitter_id','screen_name','name','description','location','friends_count','followers_count','statuses_count','created_at'])
 

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(config["twitter_api"]["consumer_key"],
                           config["twitter_api"]["consumer_secret"])
  
# set access to user's access key and access secret 
auth.set_access_token(config["twitter_api"]["access_token"], 
                      config["twitter_api"]["access_token_secret"])
  
# calling the api 
api = tweepy.API(auth)

for current_user_list in lists_of_users:
    # Use this to collect by ID:
    
    query_list = [int(numeric_string) for numeric_string in current_user_list.tolist()]
    users = api.lookup_users(user_id=query_list)

    for user in users:
        d_df = d_df.append({
            'twitter_id':user.id,
            'screen_name':user.screen_name, 
            'name':user.name, 
            'description':user.description,
            'location':user.location,
            'friends_count':user.friends_count,'followers_count':user.followers_count, 'statuses_count':user.statuses_count,'created_at':user.created_at}, ignore_index=True)


# Use SocialVec
## Get the latest SocialVec model here:
https://github.com/nirlotan/SocialVec/tree/master/models

The below code extracts only legistlators that were originally part of SocialVec.

There is another method for getting embeddings for users 

e.g.: SocialVec_v6_sg_all.model

In [ ]:
from gensim.models import Word2Vec
SocialVec = Word2Vec.load(config["SocialVec"]["model_path"])

In [ ]:
d_df.twitter_id = d_df.twitter_id.apply(lambda x: str(int(x)))

### Merge datasets and sort by popularity

In [ ]:
d_df = d_df.merge(df,left_on='twitter_id',right_on='social.twitter_id').sort_values(by='followers_count',ascending=False)

In [ ]:
def get_embedding(str_uid):
    try:
        SocialVec.wv[str_uid]
        return ('available')
    except:
        return ('missing')

get_embedding('2001')

In [ ]:
d_df['SocialVec_availability']= d_df.twitter_id.apply( lambda x: get_embedding(x))
d_df = d_df.drop_duplicates()

if save_file:
    d_df.to_csv('legislators_twitter_with_SocialVec_availability.csv')